In [17]:
import os

print(os.getcwd())

print(os.listdir('./data/lahman_1871-2023_csv/'))

file_path = './data/lahman_1871-2023_csv/Pitching.csv'
df = pd.read_csv(file_path)

/Users/jacobgareis/Downloads/DSCI326/DSCI326/Lab1.1
['AwardsManagers.csv', 'Managers.csv', 'AwardsPlayers.csv', 'Fielding.csv', 'Salaries.csv', 'Parks.csv', 'Schools.csv', 'People.csv', 'PitchingPost.csv', 'Teams.csv', 'Appearances.csv', 'AwardsSharePlayers.csv', 'TeamsFranchises.csv', 'Batting.csv', 'ManagersHalf.csv', 'FieldingOF.csv', 'Pitching.csv', 'CollegePlaying.csv', 'HomeGames.csv', 'HallOfFame.csv', 'AwardsShareManagers.csv', 'BattingPost.csv', 'TeamsHalf.csv', 'SeriesPost.csv', 'FieldingPost.csv', 'AllstarFull.csv', 'FieldingOFsplit.csv']


In [53]:
import polars as pl

# Load the pitching and batting data
pitching_df = pl.read_csv("./data/lahman_1871-2023_csv/Pitching.csv")
batting_df = pl.read_csv("./data/lahman_1871-2023_csv/Batting.csv")

# Filter the pitching data for years >= 1946
pitching_df = pitching_df.filter(pl.col('yearID') >= 1946)

# Group and aggregate the pitching data
grouped_pitching = pitching_df.group_by(['yearID', 'lgID']).agg([
    pl.col('R').sum().alias('Total_Runs_Allowed'),
    pl.col('IPouts').sum().alias('Total_Outs')
])

# Add DH Rule column to the pitching data
grouped_pitching = grouped_pitching.with_columns(
    pl.when(
        (pl.col('lgID') == 'AL') & (pl.col('yearID') >= 1973)
    ).then(pl.lit('Yes')).when(
        (pl.col('lgID') == 'NL') & (pl.col('yearID').is_in([2020]) | (pl.col('yearID') >= 2022))
    ).then(pl.lit('Yes')).otherwise(pl.lit('No')).alias('DH_Rule')
)

# Calculate ERA for the pitching data
grouped_pitching = grouped_pitching.with_columns(
    (pl.col('Total_Runs_Allowed') / pl.col('Total_Outs') * 27).alias('ERA')
)

# Write the grouped pitching data to a CSV file
grouped_pitching.write_csv("./data/lahman_1871-2023_csv/League_Wide_ERA.csv")

# Filter the batting data for years >= 1946
batting_df = batting_df.filter(pl.col('yearID') >= 1946)

# Group and aggregate the batting data
grouped_batting = batting_df.group_by(['yearID', 'lgID']).agg([
    pl.col('H').sum().alias('Total_Hits'),
    pl.col('2B').sum().alias('Total_Doubles'),
    pl.col('3B').sum().alias('Total_Triples'),
    pl.col('HR').sum().alias('Total_Home_Runs'),
    pl.col('AB').sum().alias('Total_At_Bats')
])

# Add DH Rule column to the batting data
grouped_batting = grouped_batting.with_columns(
    pl.when(
        (pl.col('lgID') == 'AL') & (pl.col('yearID') >= 1973)
    ).then(pl.lit('Yes')).when(
        (pl.col('lgID') == 'NL') & (pl.col('yearID').is_in([2020]) | (pl.col('yearID') >= 2022))
    ).then(pl.lit('Yes')).otherwise(pl.lit('No')).alias('DH_Rule')
)

# Calculate Slugging Percentage for the batting data
grouped_batting = grouped_batting.with_columns(
    (
        (pl.col('Total_Hits') - pl.col('Total_Doubles') - pl.col('Total_Triples') - pl.col('Total_Home_Runs')) + 
        2 * pl.col('Total_Doubles') + 
        3 * pl.col('Total_Triples') + 
        4 * pl.col('Total_Home_Runs')
    ).alias('Slugging_Percentage') / pl.col('Total_At_Bats')
)

# Write the grouped batting data to a CSV file
grouped_batting.write_csv("./data/lahman_1871-2023_csv/League_Wide_Slugging_PercentageV6.csv")

In [52]:
grouped_batting

yearID,lgID,Total_Hits,Total_Doubles,Total_Triples,Total_Home_Runs,Total_At_Bats,DH_Rule,Slugging_Percentage
i64,str,i64,i64,i64,i64,i64,str,f64
1959,"""NL""",11015,1788,324,1159,42330,"""No""",0.399906
2020,"""NL""",7245,1402,126,1161,29399,"""Yes""",0.421171
1975,"""NL""",17002,2781,458,1233,66102,"""No""",0.369096
1997,"""AL""",21171,4097,398,2477,78235,"""Yes""",0.428133
1963,"""NL""",13434,1984,439,1215,54803,"""No""",0.363867
…,…,…,…,…,…,…,…,…
1992,"""NL""",16538,2967,459,1262,65748,"""No""",0.368209
2018,"""NL""",20403,3993,457,2685,82722,"""No""",0.403339
1988,"""NL""",16277,2828,415,1279,65563,"""No""",0.362583
